In [ ]:
! pip install opencv-python

In [1]:
import os 
import cv2
import random 
import pickle 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop

In [2]:
import os
without_accd_dir = 'C:/ACCIDENT_DATASET_v1/No Accident'
with_accd_dir = 'C:/ACCIDENT_DATASET_v1/Accident'
imgs_without_acc = [file for file in os.listdir(without_accd_dir) if file.endswith('.jpg')]
print('There are',len(imgs_without_acc), 'whithout accidents images')
imgs_with_acc = [file for file in os.listdir(with_accd_dir) if file.endswith('.jpg')]
print('There are',len(imgs_with_acc), 'whith accidents images')

There are 1500 whithout accidents images
There are 1357 whith accidents images


In [10]:
new_dir = 'C:/ACCIDENT_DATASET_v1/AccDetectSplited/'
os.mkdir(new_dir)
train_folder = os.path.join(new_dir, 'train')
train_accident = os.path.join(train_folder, 'accident')
train_noaccident= os.path.join(train_folder, 'no_accident')

val_folder = os.path.join(new_dir, 'validation')
val_accident = os.path.join(val_folder, 'accident')
val_noaccident = os.path.join(val_folder, 'no_accident')

os.mkdir(train_folder)
os.mkdir(train_accident)
os.mkdir(train_noaccident)

os.mkdir(val_folder)
os.mkdir(val_accident)
os.mkdir(val_noaccident)

In [11]:
# train no_accident
import shutil
imgs = imgs_without_acc[:1000]
for img in imgs:
    origin = os.path.join(without_accd_dir, img)
    destination = os.path.join(train_noaccident, img)
    shutil.copyfile(origin, destination)
    
# validation no_accident
imgs = imgs_without_acc[1000:]
for img in imgs:
    origin = os.path.join(without_accd_dir, img)
    destination = os.path.join(val_noaccident, img)
    shutil.copyfile(origin, destination)

In [12]:
# train accident
imgs = imgs_with_acc[:1000]
for img in imgs:
    origin = os.path.join(with_accd_dir, img)
    destination = os.path.join(train_accident, img)
    shutil.copyfile(origin, destination)
    
# validation accident
imgs = imgs_with_acc[1000:]
for img in imgs:
    origin = os.path.join(with_accd_dir, img)
    destination = os.path.join(val_accident, img)
    shutil.copyfile(origin, destination)

In [2]:
def create_data(CATEGORIES,DATADIR,IMG_SIZE):## chargement du dataset(ensembles des images 'with-accident' et 'without-accident')
    data = []
    for category in CATEGORIES:  
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)  
        for img in os.listdir(path):   
            try:
                img_array = cv2.imread(os.path.join(path,img))   
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                data.append([img_array, class_num])  
            except Exception as e:  
                pass
    return data 

In [3]:
#(création des données d'entrainement et du test)
IMG_SIZE = 300 
cat = ['accident','no_accident']
training_path = 'C:/ACCIDENT_DATASET_v1/AccDetectSplited/train'
testing_path = 'C:/ACCIDENT_DATASET_v1/AccDetectSplited/validation'

training_data = create_data(cat, training_path, IMG_SIZE)
testing_data = create_data(cat, testing_path, IMG_SIZE)


In [4]:
print("training_data :",len(training_data))
print("testing_data :",len(testing_data))

training_data : 2000
testing_data : 857


In [5]:
#séparation des images et leurs labels
def lebalization(data):
    random.shuffle(data) 
    X = []
    y = []
    for features,label in data:
        X.append(features)
        y.append(label)
    return  np.array(X), np.array(y)

In [6]:
X_train, y_train = lebalization(training_data)
X_test, y_test = lebalization(testing_data)

In [7]:
print(type(X_train))

<class 'numpy.ndarray'>


In [8]:
#normalization des images 
X_train =X_train.reshape(-1, IMG_SIZE, IMG_SIZE,3)
X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [9]:
X_train.shape

(2000, 300, 300, 3)

In [10]:
X_train1 = X_train[:700] / 255 

In [11]:
X_train2 = X_train[700:1400] / 255 

In [12]:
X_train3 = X_train[1400:] / 255 

In [13]:
X_train0=np.concatenate((X_train1,X_train2))

In [14]:
X_train=np.concatenate((X_train0,X_train3))

In [15]:
X_test = X_test / 255 

In [16]:
X_test.shape

(857, 300, 300, 3)

In [17]:
#arret de l'entrainement en cas de l'accuracy >0.99
class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')>0.991):
            print ("\nReached 99% of accuracy")
            self.model.stop_training=True       

In [18]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 1 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    # The fourth convolution  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Dropout(0.5),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
]) 
 

In [19]:
#model.compile(loss='binary_crossentropy',
#              optimizer=RMSprop( learning_rate=1e-4),
#              metrics=['accuracy'])

In [20]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [23]:
earlyStop = myCallbacks()
history = model.fit(X_train, y_train, epochs=50,callbacks=[earlyStop])

Epoch 1/50
63/63 [==============================] - 141s 2s/step - loss: 0.6517 - accuracy: 0.6115
Epoch 2/50
63/63 [==============================] - 113s 2s/step - loss: 0.5292 - accuracy: 0.7240
Epoch 3/50
63/63 [==============================] - 111s 2s/step - loss: 0.4865 - accuracy: 0.7545
Epoch 4/50
63/63 [==============================] - 112s 2s/step - loss: 0.4313 - accuracy: 0.8010
Epoch 5/50
63/63 [==============================] - 114s 2s/step - loss: 0.3929 - accuracy: 0.8225
Epoch 6/50
63/63 [==============================] - 113s 2s/step - loss: 0.3492 - accuracy: 0.8505
Epoch 7/50
63/63 [==============================] - 113s 2s/step - loss: 0.2870 - accuracy: 0.8775
Epoch 8/50
63/63 [==============================] - 112s 2s/step - loss: 0.2298 - accuracy: 0.9050
Epoch 9/50
63/63 [==============================] - 109s 2s/step - loss: 0.2054 - accuracy: 0.9180
Epoch 10/50
63/63 [==============================] - 109s 2s/step - loss: 0.1458 - accuracy: 0.9425
Epoch 11/

In [ ]:
#evaluation du modèle
model.evaluate(X_test, y_test)

In [ ]:
import sklearn.metrics as metrics
probs = model.predict(X_test)
#preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, probs)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show() 

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
#ax1.plot(history.history['loss'], color='b', label="Training loss")
#ax1.plot(history.history['val_loss'], color='r', label="validation loss")
#ax1.set_xticks(np.arange(1, 21, 1))
#ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="training accuracy")
ax2.plot(history.history['loss'], color='r',label="training loss")
ax2.set_xticks(np.arange(1, 22, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
#prediction en utilisant le dataset du test
y_predicted = model.predict(X_test)

In [ ]:
y_predicted[6]

In [ ]:
plt.imshow(X_test[6].reshape(300,300,3)) 
plt.show()

In [ ]:
y_predicted_flattened=y_predicted.reshape(-1,)
y_predicted_flattened[:8]

In [ ]:
y_predicted_labels =[]
for i in y_predicted_flattened:
    if i>0.5:
        y_predicted_labels.append(1)
    else:
        y_predicted_labels.append(0)

In [ ]:
import itertools
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import numpy as np

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] 
    else:
        1 
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Function to calculate accuracy 
def cal_accuracy(model,X_test,y_test): 
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)
    print ("Accuracy : ", round(accuracy_score(y_test,y_pred)*100,2))
    print ("Precision : ", round(precision_score(y_test,y_pred)*100,2))
    print ("Sensitivity / Recall : ", round(recall_score(y_test,y_pred)*100,2))
    print ("F1_score : ", round(f1_score(y_test,y_pred)*100,2)) 
    cnf_matrix_tra = confusion_matrix(y_test, y_pred)
    class_names = [0,1]
    plt.figure()
    plot_confusion_matrix(cnf_matrix_tra , classes=class_names, title='Confusion matrix')
    plt.show()
    print("Report : ") 
    print(classification_report(digits=6,y_true=y_test, y_pred=y_pred)) 

In [ ]:
cal_accuracy(model,X_test, y_test)

In [ ]:
url=input('Enter URL of Image :')

#img=imread(url)
img=cv2.imread(url,3)
plt.imshow(img) 
img_resize = cv2.resize(img, (300, 300))  
new_img = img_resize.reshape(-1, IMG_SIZE, IMG_SIZE, 3) 

y_predicted = model.predict(new_img)
print(y_predicted[0]) 
print("car",cat[int(y_predicted[0])]) 

In [ ]:
y_predicted[0]

In [ ]:
model.save('Model_CNN_ACCIDENT_DATASET_v3_93%.h5')